In [1]:
import utils
import pandas as pd
import importlib
import expert_factory
importlib.reload(expert_factory)
import numpy as np

In [2]:
def calcula_verossimilhanca(num_experts, y_true, output_experts, output_gating):
    py = np.zeros(shape=(len(y_true),num_experts))
    for expert in range(0,num_experts):
        for index,y in enumerate(y_true):
            #calcula a diff entre o real e o que o cada expert previu
            #diff = Ytr(j,:)-Yaux(j,:);
            diff = y - output_experts[index,expert]
            #Py(j,i)=exp(-diff*diff'/(2*var(i)));   
            py[index,expert] = np.exp(-np.dot(diff, diff.T) / (2))
    # Likelihood= sum(log(sum(Yg.*Py,2)));
    #haux = Yg.*Py;
    haux = np.multiply(output_gating, py)
    likelihood = np.sum(np.log(np.sum(haux,axis=1)))
    return likelihood,haux       

In [3]:
def output_final(X, gating_network, all_experts):
    #Calcula saida do gating
    gating_output = gating_network.feedforward(X)[-1]
    experts_outputs = []
    #calcula saida do experts
    for exp_net in all_experts:
        experts_outputs.append(exp_net.feedforward(X)[-1].tolist())
    #retorna o melhor expert como saida
    final_output = []
    for index_gate, result in enumerate(np.argmax(gating_output,axis=1)):
        final_output.append(experts_outputs[result][index_gate])
    return final_output
        

def maximiza_gating(gating_network, X, y, X_val, y_val,h):
    gating_network.train(X, y,
                         X_val, y_val,
                         numero_max_erro_val=10,
                         h_tipo='gat',
                         h=h,
                         plot=False)
    
def maximiza_expert(expert_network, X, y, X_val, y_val,h):
    expert_network.train(X, y,
                         X_val, y_val,
                         numero_max_erro_val=10,
                         h_tipo='exp',
                         h=h,
                         plot=False)

In [4]:
#Lendo o dadoa
df = pd.read_csv('data/treinamento-1.txt', header=None)

#normalizando o dado
df = utils.normalize_data(df)
num_lags = 15

#criando Lag
lagged_data = utils.create_lag(df, num_lags)
lagged_data = lagged_data.reset_index(drop=True)

X = lagged_data.drop(['y'],axis=1)
y = lagged_data['y']

#Criando conjunto de dados
fracao_dados_para_treino = 0.7
fracao_dados_para_teste = 0.2
X_train,y_train,X_test,y_test,X_val,y_val = utils.treino_teste_validacao(X,y, frac_train=fracao_dados_para_treino, frac_test=fracao_dados_para_teste)

Tamanho total 985
Tamanho treino 689
Tamanho teste 197
Tamanho validacao 99


In [5]:
num_experts = 3
all_ne = X_train.shape[1]
## Vamos criar alguns experts
all_experts = []
exp1 = expert_factory.Expert(ne=all_ne,nh=5,ns=1,g_h='linear',g_o='sigmoid',max_epoch=1000, alpha=0.1)
exp2 = expert_factory.Expert(ne=all_ne,nh=3,ns=1,g_h='tan_h',g_o='sigmoid',max_epoch=2000, alpha=0.05)
exp3 = expert_factory.Expert(ne=all_ne,nh=3,ns=1,g_h='sigmoid',g_o='sigmoid',max_epoch=2000, alpha=0.3)

all_experts.append(exp1)
all_experts.append(exp2)
all_experts.append(exp3)

In [6]:
#Criando rede gating   
gating_ne = X_train.shape[1]
gating_nh = 5
gating_ns = num_experts
gating_network = expert_factory.Expert(gating_ne,gating_nh,gating_ns, g_h='sigmoid', g_o='softmax',max_epoch=3000, alpha=0.2)

In [7]:
likelihood = 0
old_likelihood = -np.inf
iters = 0
max_iters = 100
while abs(likelihood-old_likelihood) > 1e-3 and iters < max_iters:
    iters += 1
    #calcula a saida para cada rede
    #A funcao retorna 4 variaveis, queremos apenas a ultima que representa o output (por isso o [-1])
    output_gating = gating_network.feedforward(X_train)[-1]
    output_experts = np.matrix([np.ravel(expert.feedforward(X_train)[-1]).tolist() for expert in all_experts]).T
    #Agora que temos a saida comecamos com a funcao de EM
    old_likelihood = likelihood
    #Passo E (Expectation)
    #Com os parametros atuais calculamos calculamos a 'expectation' posterior para cada expert
    likelihood,haux_train = calcula_verossimilhanca(num_experts, y_train, output_experts, output_gating)
    #likelihood_val,haux_val = calcula_verossimilhanca(num_experts, y_train, output_experts, output_gating, matriz_covariancia)
    #h = haux./(sum(haux,2)*ones(1,m));
    h = np.divide(haux_train, np.dot(np.sum(haux_train,axis=1).reshape(haux_train.shape[0],1),np.ones((1,num_experts))))
    #Passo M (Maximizacao)
    maximiza_gating(gating_network, X_train, y_train, X_val, y_val,h)
    #Itera por cada expert o treinando com seu respectivo h
    for exp_index, expert in enumerate(all_experts):
        output_individual_exp = []
        for item in h[:,0]:
            output_individual_exp.append([item])
        maximiza_expert(expert, X_train, y_train, X_val, y_val,h=output_individual_exp)
    
    saida_final = output_final(X_train, gating_network, all_experts)
    erro = saida_final - y_train 
    loss = np.square(erro).mean()
    print(loss)
    print(likelihood)

0.346537204953
-4611.08594159
0.268517024679
-4629.57387705
0.336290736738
-4626.50360151
0.364347790265
-4682.70021733
0.232909889467
-4731.35197405
0.369886646845
-4729.91607589
0.343813920034
-4825.08477515
0.186425055238
-4867.72862218
0.277257877394
-4868.41087199
0.265027379507
-4957.58885956
0.252772327309
-5014.00396925
0.244824641766
-5072.95102196
0.246046734177
-5135.75015327
0.25890915395
-5203.92760598
0.283454426728
-5278.20418611
0.278794149698
-5358.47917114
0.275199699297
-5430.56326426
0.272558677328
-8294.11837919
0.270652311172
-8398.63574567
0.269291542529
-8503.90202546
0.268325601957
-8609.83456515
0.26764023787
-8716.36465394
0.267151732489
-8823.43688208
0.266800105823
-8931.00704888
0.266543034307
-9039.03979206
0.266350963839
-9147.50646433
0.266203407998
-9256.38342239
0.266086241677
-9365.65072526
0.265989766001
-9475.29117645
0.265907344473
-9585.28963283
0.265834450744
-9695.63251117
0.265768007986
-9806.30743749
0.265705932524
-9917.3029976
0.26564681954

In [10]:
import matplotlib.pyplot as plt